In [1]:
import tensorflow as tf
import numpy as np
import tensorflow_datasets as tfds
from abc import ABC, abstractmethod
from sklearn.ensemble import RandomForestClassifier

The goal is to build a DigitClassifier that takes an algorithm as an input parameter. Possible values for the algorithm are: cnn, rf, rand.

There are 5 classes:
- **DigitClassificationInterface** sets the structure of how each model class should look like.
- **CNNClassifier** contains CNN model, preprocess function, and prediction function.
- **RFClassifier** contains RandomForest model, preprocess function, and prediction function.
- **RandomClassifier** contains no model it has preprocess function, and random prediction function.
- **DigitClassifier** puts everything together to define algorithms and sets predictions.

In [2]:
class DigitClassificationInterface(ABC):
    # Abstract base class for digit classification models
    
    @abstractmethod
    def preprocess_input(self, image: np.ndarray) -> np.ndarray:
      # Preprocess the input image
      pass

    @abstractmethod
    def predict(self, image: np.ndarray) -> int:
      # Predict outcome
      pass

class CNNClassifier(DigitClassificationInterface):
    # CNN classifier class

    def __init__(self):
        self.model = self.create_model()

    def create_model(self) -> tf.keras.Model:
        # Create and compile the model
        model = tf.keras.Sequential()
        model.add(tf.keras.Input(shape=(28, 28, 1)))
        model.add(tf.keras.layers.Conv2D(32, (3, 3), activation='relu'))
        model.add(tf.keras.layers.MaxPooling2D((2, 2)))
        model.add(tf.keras.layers.Conv2D(64, (3, 3), activation='relu'))
        model.add(tf.keras.layers.MaxPooling2D((2, 2)))
        model.add(tf.keras.layers.Flatten())
        model.add(tf.keras.layers.Dense(64, activation='relu'))
        model.add(tf.keras.layers.Dense(10))
        model.compile(
          optimizer='adam',
          loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
          metrics=['accuracy']
        )
        return model

    def preprocess_input(self, image: np.ndarray) -> np.ndarray:    
        # Preprocess image for CNN calssifier
        # Scale [0,1]
        image = tf.cast(image, tf.float32) / 255.0
        # Add batch dimension (originally dim is (28,28,1))
        if len(image.shape) == 3:
            image = tf.expand_dims(image, 0)
        return image

    def predict(self, image: np.ndarray) -> int:
        # Predict outcome using CNN
        preprocessed = self.preprocess_input(image)
        predictions = self.model(preprocessed, training=False)
        return int(tf.argmax(predictions[0]))

class RFClassifier(DigitClassificationInterface):
    # Random Forest classifier class

    def __init__(self):
        self.model = RandomForestClassifier(n_estimators=100)
        self.fit()

    # Had to add this method because RFClassifier doesn't work without actually fitting model on some data
    def fit(self):
        # Take 100 images from train set to fit model
        ds_train = tfds.load('mnist', split='train[:100]', as_supervised=True)
        np_iterator = tfds.as_numpy(ds_train)
        # Split images and labels into separate lists
        X_train = []
        Y_train = []
        for image, label in np_iterator:
            X_train.append(image.reshape(-1))  # Flatten each image into 1D vector
            Y_train.append(label)

        X_train = np.array(X_train)
        Y_train = np.array(Y_train)
        self.model.fit(X_train, Y_train)

    def preprocess_input(self, image: np.ndarray) -> np.ndarray:
        # Flatten image
        # Ensure correct shape and flatten
        if len(image.shape) == 4:
            return image.reshape(image.shape[0], -1)
        return image.reshape(1, -1)

    def predict(self, image: np.ndarray) -> int:
        # Predict outcome using Random Forest
        preprocessed = self.preprocess_input(image)
        return int(self.model.predict(preprocessed)[0])

class RandomClassifier(DigitClassificationInterface):
    # Random classifier class

    def preprocess_input(self, image: np.ndarray) -> np.ndarray:
        # Ensure correct shape
        if len(image.shape) == 4:
            image = image[0]  # Take first image if batch

        # Calculate center crop coordinates
        h_center = image.shape[0] // 2
        w_center = image.shape[1] // 2
        return image[h_center-5:h_center+5, w_center-5:w_center+5]

    def predict(self, image: np.ndarray) -> int:
        # Predict random outcome
        _ = self.preprocess_input(image) 
        return np.random.randint(0, 10) # Return random number for classification

class DigitClassifier:
    # Main classifier that puts everything together

    def __init__(self, algorithm: str):
        # Choose classifier
        if algorithm == "cnn":
            self.model = CNNClassifier()
        elif algorithm == "rf":
            self.model = RFClassifier()
        elif algorithm == "rand":
            self.model = RandomClassifier()
        else:
            raise ValueError(f"Unknown algorithm: {algorithm}")

    def predict(self, image: np.ndarray) -> int:
        # Make prediction
        return self.model.predict(image)

In [3]:
def test_classifier(algorithm: str, test_images: np.ndarray, test_labels: np.ndarray) -> list:
    # Create classifier using specified algorithm
    classifier = DigitClassifier(algorithm)
    
    # Make predictions
    predictions = []
    for image in test_images:
        pred = classifier.predict(image)
        predictions.append(pred)
    
    # Print results
    print(f"\n{algorithm} classifier:")
    print(f"true labels:     {test_labels}")
    print(f"predictions:     {predictions}")
    return predictions

### Example of usage

In [5]:
# Load 5 images from test set
ds_test = tfds.load('mnist', split='test[:5]', as_supervised=True)
ds_test = ds_test.batch(5) 
np_iterator = tfds.as_numpy(ds_test)

test_images, test_labels = next(iter(np_iterator))

# Make forecast
cnn_predictions = test_classifier('cnn', test_images, test_labels)
rf_predictions = test_classifier('rf', test_images, test_labels)
rand_predictions = test_classifier('rand', test_images, test_labels)


cnn classifier:
true labels:     [2 0 4 8 7]
predictions:     [0, 6, 6, 3, 5]

rf classifier:
true labels:     [2 0 4 8 7]
predictions:     [9, 0, 4, 1, 7]

rand classifier:
true labels:     [2 0 4 8 7]
predictions:     [8, 1, 4, 7, 1]


2024-12-04 20:53:19.911307: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
